In [1]:
import pymongo

In [2]:
# 데이터베이스 연결
connection = pymongo.MongoClient('localhost', 27017)
client = pymongo.MongoClient()
print(client)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)


In [3]:
# DB 목록
client.list_database_names()

['admin', 'config', 'local', 'uspto']

In [4]:
# Collection (테이블) 목록
client.uspto.list_collection_names()

['citations', 'citations_mini', 'USPAT_mini', 'USPAT']

In [5]:
# USPAT collection에 저장된 document 출력
db = client["uspto"]
collection = db["USPAT"]
collection.find_one()

{'_id': ObjectId('640448527058e9b008e7c21b'),
 'guid': 'US-11565042-B1',
 'publicationReferenceDocumentNumber': '11565042',
 'compositeId': '1000000383486!US-US-11565042',
 'publicationReferenceDocumentNumber1': '11565042',
 'datePublishedKwicHits': None,
 'datePublished': '2023-01-31T00:00:00Z',
 'inventionTitle': 'Anesthesia and/or sedation system and method',
 'type': 'USPAT',
 'mainClassificationCode': '1/1',
 'applicantName': ['Bibian; Stéphane', 'Zikov; Tatjana', 'Barua; Sankar'],
 'assigneeName': ['NeuroWave Systems Inc.'],
 'uspcFullClassificationFlattened': None,
 'ipcCodeFlattened': 'A61B5/369;A61M5/172;A61M5/142',
 'cpcInventiveFlattened': 'A61M5/1723;A61B5/369',
 'cpcAdditionalFlattened': 'A61M2005/14296;A61M2005/14208',
 'applicationFilingDate': ['2013-08-08T00:00:00Z'],
 'applicationFilingDateKwicHits': None,
 'relatedApplFilingDate': ['2012-08-08T00:00:00Z'],
 'primaryExaminer': 'Price; Nathan R',
 'assistantExaminer': ['Frehe; William'],
 'applicationNumber': '13/962565

In [6]:
# citation collection에 저장된 document 출력
collection = db["citations"]
collection.find_one()

{'_id': ObjectId('64097a1b01e2bf0d26e37fd8'),
 'cited': '5741211',
 'citing': '11565042'}

In [7]:
# 'country' 컬럼이 'US'인 특허 추출
for i, doc in enumerate(db.USPAT.find({'country': 'US'}, {'guid': 1, 'country': 1, '_id': 0})):
    print(doc)
    if i > 10: break
        
# [SQL]
# SELECT guid, country
# FROM USPAT
# WHERE country = 'US';

{'guid': 'US-11565042-B1', 'country': 'US'}
{'guid': 'US-11565025-B1', 'country': 'US'}
{'guid': 'US-11565470-B1', 'country': 'US'}
{'guid': 'US-11568442-B1', 'country': 'US'}
{'guid': 'US-11568982-B1', 'country': 'US'}
{'guid': 'US-11568388-B1', 'country': 'US'}
{'guid': 'US-11568389-B1', 'country': 'US'}
{'guid': 'US-11568038-B1', 'country': 'US'}
{'guid': 'US-11568446-B1', 'country': 'US'}
{'guid': 'US-11568377-B1', 'country': 'US'}
{'guid': 'US-11568301-B1', 'country': 'US'}
{'guid': 'US-11568863-B1', 'country': 'US'}


In [8]:
# IPC에 G06F가 포함된 특허 추출
for i, doc in enumerate(db.USPAT.find({'ipcCodeFlattened': {"$regex": 'G06F'}}, 
                                      {'ipcCodeFlattened': 1, 'inventionTitle': 1, '_id': 0}).limit(10)):
    print(doc,"\n")
    
# [SQL]
# SELECT inventionTitle, ipcCodeFlattened
# FROM USPAT
# WHERE ipcCodeFlattened LIKE '%G06F%';

{'inventionTitle': 'Machine learning in resource-constrained environments', 'ipcCodeFlattened': 'G06N20/00;G06F8/65'} 

{'inventionTitle': 'Applied artificial intelligence technology for narrative generation based on explanation communication goals', 'ipcCodeFlattened': 'G06F40/30;G06F40/295'} 

{'inventionTitle': 'Semi-structured data machine learning', 'ipcCodeFlattened': 'G06N20/00;G06F16/835'} 

{'inventionTitle': 'Tree-based format for data storage', 'ipcCodeFlattened': 'G06F16/22;G06F16/28'} 

{'inventionTitle': 'Virtual storage interface', 'ipcCodeFlattened': 'G06F16/188;G06Q40/00'} 

{'inventionTitle': 'Copying buckets from a remote shared storage system to memory associated with a search node for query execution', 'ipcCodeFlattened': 'G06F16/901;G06F16/2458'} 

{'inventionTitle': 'Event-based debug, trace, and profile in device with data processing engine array', 'ipcCodeFlattened': 'G06F13/10;G06F13/16'} 

{'inventionTitle': 'Blockchain network based on machine learning-based

In [9]:
pipeline = list()
pipeline.append({'$limit': 100000})
pipeline.append({'$out': 'citations_mini'})
results = db.citations.aggregate(pipeline)

In [10]:
pipeline = list()
pipeline.append({'$limit': 50000})
pipeline.append({'$out': 'USPAT_mini'})
results = db.USPAT.aggregate(pipeline)

In [11]:
# 특허 문서 수가 100개 이상인 특허 클래스 조합별 특허 문서 수와 평균 청구항 수 추출
pipeline = list()
pipeline.append({'$group': {'_id': '$ipcAllMainClassification', 'patent_Count': {'$sum': 1}, 
                            'average_Claims': {'$avg': {'$toDecimal': '$numberOfClaims'}}}})
pipeline.append({'$addFields': {'average_Claims': {'$round': ['$average_Claims', 2]}}})
pipeline.append({'$match': {'patent_Count': {'$gte': 100}}})
print(pipeline, "\n")

results = db.USPAT_mini.aggregate(pipeline)
for i, doc in enumerate(results):
    print(doc)
    if i > 10: break
        
# [SQL]
# SELECT ipcAllMainClassification, patent_Count, average_Claims
# FROM
# (
#    SELECT ipcAllMainClassification, COUNT(*) AS patent_Count, AVG(numberofClaims) AS average_Claims
#    FROM USPAT
#    GROUP BY ipcAllMainClassification
# )
# WHERE patent_Count >= 100;

[{'$group': {'_id': '$ipcAllMainClassification', 'patent_Count': {'$sum': 1}, 'average_Claims': {'$avg': {'$toDecimal': '$numberOfClaims'}}}}, {'$addFields': {'average_Claims': {'$round': ['$average_Claims', 2]}}}, {'$match': {'patent_Count': {'$gte': 100}}}] 

{'_id': ['G06Q', 'H04L', 'G06F'], 'patent_Count': 178, 'average_Claims': Decimal128('19.25')}
{'_id': ['H04L', 'H04W'], 'patent_Count': 983, 'average_Claims': Decimal128('18.55')}
{'_id': ['H04L', 'H04B'], 'patent_Count': 138, 'average_Claims': Decimal128('19.51')}
{'_id': ['H04L', 'G06F'], 'patent_Count': 1153, 'average_Claims': Decimal128('18.66')}
{'_id': ['G11C', 'H01L'], 'patent_Count': 176, 'average_Claims': Decimal128('17.53')}
{'_id': ['H04N', 'G06F'], 'patent_Count': 157, 'average_Claims': Decimal128('16.24')}
{'_id': ['H01M'], 'patent_Count': 586, 'average_Claims': Decimal128('13.31')}
{'_id': ['G03G'], 'patent_Count': 129, 'average_Claims': Decimal128('15.62')}
{'_id': ['G02B'], 'patent_Count': 176, 'average_Claims': 